In [1]:
%load_ext autoreload
%autoreload 2


import os, sys

import torch
import torchio as tio
from ipywidgets import interact
import matplotlib.pyplot as plt

dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)
from data_processing import *

In [2]:
subject_loader = ComposeLoaders([
ImageLoader(glob_pattern="whole_roi.*", image_name="whole_roi", image_constructor=tio.LabelMap,
                label_values={"left_whole": 1, "right_whole": 2}
                ),
])

cohorts = {'all': RequireAttributes(['whole_roi'])}
dataset_path = "X:/Datasets/Diffusion_MRI/"
dataset = SubjectFolder(root=dataset_path, subject_path="subjects", cohorts=cohorts, subject_loader=subject_loader)

In [ ]:
from skimage.measure import marching_cubes
import meshio

image_name = 'whole_roi'
for subject in dataset:
    x = subject[image_name].data[0]

    W, H, D = x.shape
    half_W = W // 2

    left_x = x[:half_W]
    right_x = x[half_W:]

    left_verts, left_faces, left_normals, left_values = marching_cubes(left_x.numpy())
    right_verts, right_faces, right_normals, right_values = marching_cubes(right_x.numpy())

    left_mesh = meshio.Mesh(left_verts, [("triangle", left_faces)])
    right_mesh = meshio.Mesh(right_verts, [("triangle", right_faces)])

    out_path = os.path.join(dataset_path, "shape", subject['name'])
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    left_mesh.write(os.path.join(out_path, f"left_{image_name}.obj"))
    right_mesh.write(os.path.join(out_path, f"right_{image_name}.obj"))

In [ ]:
import pygalmesh

image_name = 'whole_roi'
for subject in dataset:
    print(subject['name'])
    x = subject[image_name].data[0]

    W, H, D = x.shape
    half_W = W // 2

    left_x = (x[:half_W] > 0).to(torch.uint8)
    right_x = (x[half_W:] > 0).to(torch.uint8)
    
    # See 4.2.2 https://doc.cgal.org/latest/Mesh_3/index.html
    # and https://github.com/nschloe/pygalmesh/blob/dc2bd6c920c36cf27f5c89738bbe9488e3c4fe43/pygalmesh/main.py#L428
    params = {'facet_distance': 1., 'cell_size': 1.}
    voxel_size = (1., 1., 1.)
    left_mesh = pygalmesh.generate_from_array(left_x.numpy(), voxel_size, **params)
    right_mesh = pygalmesh.generate_from_array(right_x.numpy(), voxel_size, **params)

    out_path = os.path.join(dataset_path, "pygalmesh", subject['name'])
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    left_mesh.write(os.path.join(out_path, f"left_{image_name}.vtk"))
    right_mesh.write(os.path.join(out_path, f"right_{image_name}.vtk"))

In [6]:
import numpy as np
import pyvista as pv
import pygalmesh

subject_names = [subject['name'] for subject in dataset.all_subjects]

@interact(subject_name=subject_names)
def show(subject_name):
    subject = dataset[subject_name]
    
    @interact(facet_distance=(0., 2.), cell_size=(0., 2.))
    def choose_params(facet_distance, cell_size):
        x = subject['whole_roi'].data[0]

        W, H, D = x.shape
        half_W = W // 2

        left_x = (x[:half_W] > 0).to(torch.uint8)
        right_x = (x[half_W:] > 0).to(torch.uint8)

        params = {'facet_distance': facet_distance, 'cell_size': cell_size}
        voxel_size = (1., 1., 1.)
        left_mesh = pygalmesh.generate_from_array(left_x.numpy(), voxel_size, **params)
        right_mesh = pygalmesh.generate_from_array(right_x.numpy(), voxel_size, **params)

        plotter = pv.Plotter(notebook=True)
        mesh = pv.wrap(left_mesh)
        plotter.add_mesh(mesh)
        plotter.show(jupyter_backend='ipygany')

interactive(children=(Dropdown(description='subject_name', options=('ab300_001', 'ab300_002', 'ab300_004', 'ab…